In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

In [2]:
# 1. 범용 섹션 추출 함수
def extract_section(url, section_title):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.text, "html.parser")

        title_tag = soup.find(lambda tag: tag.name in ['strong', 'h3'] and section_title in tag.text)
        if not title_tag:
            return None

        contents = []
        for sibling in title_tag.find_next_siblings():
            if sibling.name in ['strong', 'h3'] and section_title not in sibling.text:
                break
            if sibling.name in ['p', 'ul', 'dd']:
                text = sibling.get_text(separator=" ", strip=True).replace("\n", " ")
                if text:
                    contents.append(text)

        return " ".join(contents).strip() if contents else None
    except Exception as e:
        print(f"Error processing {url} ({section_title}): {e}")
        return None

In [3]:
# def extract_departments(url):
#     headers = {"User-Agent": "Mozilla/5.0"}
#     try:
#         response = requests.get(url, headers=headers, timeout=10)
#         if response.status_code != 200:
#             return None
#         soup = BeautifulSoup(response.text, "html.parser")
#         dept_row = soup.find("th", string=lambda x: x and "진료과" in x)
#         if not dept_row:
#             return None
#         td = dept_row.find_next_sibling("td")
#         if not td:
#             return None
#         dept_list = [a.text.strip() for a in td.find_all("a") if a.text.strip()]
#         return ", ".join(dept_list) if dept_list else None
#     except Exception as e:
#         print(f"Error processing {url} (진료과): {e}")
#         return None

In [4]:
def extract_departments(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return ""
        soup = BeautifulSoup(response.text, "html.parser")
        dept_row = soup.find("th", string=lambda x: x and "진료과" in x)
        if not dept_row:
            return ""
        td = dept_row.find_next_sibling("td")
        if not td:
            return ""

        # 1. <a> 태그가 있을 경우
        a_tags = td.find_all("a")
        if a_tags:
            dept_list = [a.text.strip() for a in a_tags if a.text.strip()]
            return ", ".join(dept_list)

        # 2. <a> 태그 없고 그냥 텍스트만 있을 경우
        return td.get_text(separator=",", strip=True).replace("\n", " ")

    except Exception as e:
        print(f"Error processing {url} (진료과): {e}")
        return ""

In [5]:
df = pd.read_csv("naver_health_terms.csv")

In [6]:
# 추출할 항목
section_names = ["정의", "원인", "증상", "진단/검사", "치료"]

# 결과 저장 리스트
results = []

# 전체 실행
for _, row in tqdm(df.iterrows(), total=len(df)):
    url = row["링크"]
    disease_name = row["질환명"]

    result = {"병명": disease_name}

    for section in section_names:
        result[section] = extract_section(url, section)

    result["진료과"] = extract_departments(url)
    results.append(result)

    # print(disease_name)
    time.sleep(0.5)

100%|██████████| 1762/1762 [57:06<00:00,  1.94s/it] 


In [10]:
results

[{'병명': '가드네렐라 바지날리스 [Gardnerella Vaginalis]',
  '정의': '그람 염색 변이성(Gram-variable-staining), 혐기성 박테리아로 세균성 질염 을 일으키는 가장 흔한 원인균이다. 질내 정상 세균총의 일부로 존재하며 이 Gardnerella Vaginalis 와 같은 혐기성 세균은 질내 정상 세균총의 1% 미만을 차지한다. 그러나 질내 정상 세균총에 변화가 오며 과산화수소(hydrogen peroxide)를 형성하여 질내 세균총을 유지하는 Lactobacilli가 감소하고 Gardnerella Vaginalis와 같은 혐기성 세균이 과증식하게 되면 심한 비린내를 동반한 질분비물 증가를 호소하는 세균성 질염이 발생하게 된다.',
  '원인': None,
  '증상': None,
  '진단/검사': None,
  '치료': None,
  '진료과': '산부인과, 감염내과'},
 {'병명': '가랑이통증 [Perineal pain]',
  '정의': '넓게는 외음부-여성의 바깥 생식기관-불두덩(치골구), 대음순, 소음순, 음핵, 처녀막, 요도구, 질어귀(질전정), 질어귀망울(질전정구), 바르톨린샘, 샅부위(회음부)에 발생하는 통증을, 좁게는 후음순소대와 항문사이의 샅부위(회음부)에 발생하는 통증을 일컫는다.',
  '원인': '요로감염이나 항문 농양 등의 감염, 해당 부위의 양성 또는 악성 종양, 관련 부위의 수술후 발생하는 흉터, 유착이나 누공, 피부 병변, 치핵, 말안장 손상(Straddle injury), 음부신경포착증후군, 당뇨등에 의한 신경 손상, 골반바닥근육의 이상, 위장관계 질환(충수돌기염, 대장염 등)에 의한 연관통, 원인을 알 수 없는 외음부통증(Vulvodynia) 등.',
  '증상': '원인에 따라 증상에 다소 차이가 있을 수 있다. *요로감염: 배뇨통 및 빈뇨, 절박뇨 , 골반부위의 둔통 등 *항문주위농양: 발적과 종창, 압통 및 욱신거리는 통증을 동반한 덩이(감염에 의한 경우 열 및 오한을 동

In [13]:
# DataFrame 생성 및 저장
df_result = pd.DataFrame(results)

# "증상"이 없는 행 제거
df_result = df_result[~df_result['증상'].isnull() & (df_result['증상'].str.strip() != '')].reset_index(drop=True)

df_result.to_csv("snu_health_more_cleaned.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료")

CSV 저장 완료
